In [11]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json
from datetime import datetime

from pathlib import Path
#from midas import m
#from datetime import datetime, date
#import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/RevMed_Methylation_Status_Score_udpate_acrossAllSOWs_03112025"

## Data preocessing

In [4]:
# Readin all the SOW cumulative reports
folder_path = f"{work_dir}/RevMed_cumulative_SOWs"

# List to store the dataframes and path
dfs = []
pathes = []
files_with_BIP161 = []

# Iterate through all subfolders in the main folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            pathes.append(file_path)
            try:
                # Read the CSV file
                df = pd.read_csv(file_path, header=1)
                dfs.append(df)
                # # Identify the batch that used Sirius 1.6.1, the list is from later step
                # if df['GHSampleID'].isin(samples_BIP161_list).any():
                #     files_with_BIP161.append(file_path)
            except pd.errors.ParserError:
                print(f"Error reading file: {file_path} - possibly incorrect header.")
            except Exception as e:
                print(f"An unexpected error occurred with file: {file_path} - {str(e)}")

# Concatenate all dataframes into one final dataframe
df_ori = pd.concat(dfs, ignore_index=True)

print("Total batch # of RevMed is " + str(len(pathes)) + "\n") 
print("Total sample size of RevMed is " + str(df_ori.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of RevMed is " + str(df_ori.Patient_ID.nunique()) + "\n")

df_ori.head()

Total batch # of RevMed is 6

Total sample size of RevMed is 2068

Total unique patient sample size of RevMed is 1066



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,RMC-GI-102,GI-102C-195-1013_C2D1,A1245021,B00888215,GI-102C-195-1013,C2D1,1,SUCCESS,NaN,SNV,NaN,EMSY,11,76255601.0,19,K1003T,A>C,c.3008A>C,NM_020193.5,52.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs780165176,NaN,NaN,NaN,NaN,2635.0,2.92,NaN,NaN,Detected,0.000717,NaN,Not evaluable,Not Detected,15.18,2.5,0.0,12/17/24,11/19/2024,1/17/25,NaN,Not Applicable,Not Applicable
1,RMC-GI-102,GI-102C-195-1013_C2D1,A1245021,B00888215,GI-102C-195-1013,C2D1,1,SUCCESS,NaN,SNV,NaN,FGF8,10,103534610.0,4,T72T,C>T,c.216G>A,NM_033163.5,50.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs560812676,NaN,NaN,NaN,NaN,2558.0,2.92,NaN,NaN,Detected,0.000717,NaN,Not evaluable,Not Detected,15.18,2.5,0.0,12/17/24,11/19/2024,1/17/25,NaN,Not Applicable,Not Applicable
2,RMC-GI-102,GI-102C-195-1013_C2D1,A1245021,B00888215,GI-102C-195-1013,C2D1,1,SUCCESS,NaN,SNV,NaN,CSF1R,5,149459839.0,4,A123G,G>C,c.368C>G,NM_005211.4,50.35,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs140924076,Benign,749341.0,NaN,NaN,2302.0,2.92,NaN,NaN,Detected,0.000717,NaN,Not evaluable,Not Detected,15.18,2.5,0.0,12/17/24,11/19/2024,1/17/25,NaN,Not Applicable,Not Applicable
3,RMC-GI-102,GI-102C-195-1013_C2D1,A1245021,B00888215,GI-102C-195-1013,C2D1,1,SUCCESS,NaN,SNV,NaN,AKT2,19,40761070.0,4,D94D,G>A,c.282C>T,NM_001626.6,49.77,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs139125633,Likely_benign,470212.0,NaN,NaN,2615.0,2.92,NaN,NaN,Detected,0.000717,NaN,Not evaluable,Not Detected,15.18,2.5,0.0,12/17/24,11/19/2024,1/17/25,NaN,Not Applicable,Not Applicable
4,RMC-GI-102,GI-102C-195-1013_C2D1,A1245021,B00888215,GI-102C-195-1013,C2D1,1,SUCCESS,NaN,SNV,NaN,RHOB,2,20647235.0,1,A3A,C>T,c.9C>T,NM_004040.4,49.35,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145485033,NaN,NaN,NaN,NaN,734.0,2.92,NaN,NaN,Detected,0.000717,NaN,Not evaluable,Not Detected,15.18,2.5,0.0,12/17/24,11/19/2024,1/17/25,NaN,Not Applicable,Not Applicable


In [5]:
# Remove duplication
df_dedup = df_ori.drop_duplicates(subset = "GHSampleID")
df_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,RMC-GI-102,GI-102C-195-1013_C2D1,A1245021,B00888215,GI-102C-195-1013,C2D1,1,SUCCESS,NaN,SNV,NaN,EMSY,11,76255601.0,19,K1003T,A>C,c.3008A>C,NM_020193.5,52.72,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs780165176,NaN,NaN,NaN,NaN,2635.0,2.92,NaN,NaN,Detected,0.000717,NaN,Not evaluable,Not Detected,15.18,2.5,0.0,12/17/24,11/19/2024,1/17/25,NaN,Not Applicable,Not Applicable
32,RMC-GI-102,GI-102A-095-1019_C2D1,A1245022,B00888213,GI-102A-095-1019,C2D1,1,SUCCESS,NaN,SNV,NaN,ZNF217,20,52192551.0,4,K918Q,T>G,c.2752A>C,NM_006526.3,52.26,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs775951797,NaN,NaN,NaN,NaN,4716.0,3.92,NaN,NaN,Detected,0.000992,NaN,Not evaluable,Not Detected,31.76,3.0,1.0,12/17/24,11/27/2024,1/17/25,NaN,Not Applicable,Not Applicable
50,RMC-GI-102,GI-102A-014-1012_C2D1,A1245023,B00888219,GI-102A-014-1012,C2D1,1,SUCCESS,NaN,SNV,NaN,MAP3K1,5,56111633.0,1,L78P,TC>CT,c.233_234delTCinsCT,NM_005921.2,51.37,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Benign,521577.0,NaN,NaN,471.0,1.01,NaN,NaN,Detected,0.003127,13.05,Low,Not Detected,32.70,3.0,1.0,12/17/24,11/14/2024,1/17/25,NaN,Not Applicable,Not Applicable
83,RMC-GI-102,GI-102C-193-1021_C2D1,A1245024,B00888216,GI-102C-193-1021,C2D1,1,SUCCESS,NaN,SNV,NaN,RBM10,X,47044849.0,20,P725P,G>T,c.2175G>T,NM_005676.5,100.00,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs137940243,NaN,NaN,NaN,NaN,2192.0,0.85,NaN,NaN,Detected,0.000841,5.73,Low,Not Detected,286.89,3.0,0.0,12/17/24,12/05/2024,1/17/25,NaN,Not Applicable,Not Applicable
142,RMC-GI-102,GI-102C-204-1022_C2D1,A1245025,B00888226,GI-102C-204-1022,C2D1,1,SUCCESS,NaN,SNV,NaN,CUX1,7,101840527.0,15,Q612H,G>C,c.1836G>C,NM_181552.4,50.20,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3232.0,0.38,NaN,NaN,Detected,0.003276,5.93,Low,Not Detected,41.68,3.0,1.0,12/17/24,12/04/2024,1/17/25,NaN,Not Applicable,Not Applicable


In [6]:
# Pull "runid" from ghdb
df_1 = df_dedup.copy()

# Define the IDs you want to query for
ids = tuple(df_1['GHSampleID'].unique())

# Generate the SQL query string
query = "SELECT * FROM gh_sample WHERE run_sample_id IN {}".format(ids)

# Get the dataframe with "runid"
sample_df = functions.run_sql_query(query)
sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# Extract the date from runid and convert to datetime format for accurate comparison
sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
df_1 = df_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
df_1 = df_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
df_1 = df_1.drop(columns='run_date')

print("Total sample size of the Infinity batch after merging runid is " + str(df_1.shape[0])) 
df_1.head()


Total sample size of the Infinity batch after merging runid is 2068


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
1140,RMC-6236-001,01731440640,A0778825,B00426147,6236-001-014-0066,C1D1,0,FAIL,Low Diversity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,6/9/23,04/05/2023,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01902_0167_BH5VT7DSX7
1136,RMC-6236-001,01730340213,A0778824,B00426148,6236-001-087-0051,C1D1,1,SUCCESS,NaN,SNV,NaN,ATRX,X,76909666.0,14,E1413E,T>C,c.4239A>G,NM_000489.6,100.00,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs141974120,Benign/Likely_benign,274280.0,NaN,NaN,2101.0,0.470000,NaN,NaN,Detected,0.000878,NaN,Not evaluable,Not Detected,NaN,4.0,0.0,6/9/23,01/18/2023,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01902_0167_BH5VT7DSX7
1135,RMC-6236-001,02424040043,A0778829,B00426149,6236-001-088-0017,C1D1,1,SUCCESS,NaN,SNV,NaN,ULBP3,6,150390171.0,1,P11L,G>A,c.32C>T,NM_024518.3,53.54,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs377740812,NaN,NaN,NaN,NaN,1645.0,3.630000,NaN,NaN,Detected,0.001415,10.4,Low,Not Detected,NaN,3.5,0.0,6/9/23,10/05/2022,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01902_0167_BH5VT7DSX7
1143,RMC-6236-001,01731650681,A0778827,B00426151,6236-001-095-0070,C1D1,1,SUCCESS,NaN,SNV,NaN,RIF1,2,152276793.0,6,K198R,A>G,c.593A>G,NM_001177665.2,53.44,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2699.0,0.300000,NaN,NaN,Detected,0.002850,NaN,Not evaluable,Not Detected,NaN,3.5,0.0,6/9/23,04/20/2023,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01902_0167_BH5VT7DSX7
1142,RMC-6236-001,02431350022,A0778837,B00426152,6236-001-088-0067,C1D1,1,SUCCESS,NaN,SNV,NaN,FLT4,5,180030313.0,30,R1324P,C>G,c.3971G>C,NM_182925.4,52.37,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV56098908,rs307821,Likely_benign,251881.0,NaN,NaN,5051.0,27.922479,NaN,NaN,Detected,0.085692,29.3,High,Not Detected,NaN,4.0,0.0,6/9/23,03/30/2023,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01907_0107_AH5VTLDSX7


In [7]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        if folders:
            for folder in folders:
                primary_fc_dir = folder
                # Check if there is a 'sirius' folder under the primary fc_dir. This is for Sirius 1.5 folder structure change.
                if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                    primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
                df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_2, df_no_found = add_fc_dir_column(df_1)

print("Total sample size of the Infinity batch with fc_dir is " + str(df_2.shape[0]) + "\n") 
print("Total sample size of the Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_2.head()



Total sample size of the Infinity batch with fc_dir is 2068

Total sample size of the Infinity batch with NO fc_dir is 152



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
1140,RMC-6236-001,01731440640,A0778825,B00426147,6236-001-014-0066,C1D1,0,FAIL,Low Diversity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,6/9/23,04/05/2023,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01902_0167_BH5VT7DSX7,/ghsfa/ivd/flowcentral/230629_A01902_0167_BH5VT7DSX7.8efb7722-560b-43a3-b3db-54cf45f51d00.20230711185414
1136,RMC-6236-001,01730340213,A0778824,B00426148,6236-001-087-0051,C1D1,1,SUCCESS,NaN,SNV,NaN,ATRX,X,76909666.0,14,E1413E,T>C,c.4239A>G,NM_000489.6,100.00,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs141974120,Benign/Likely_benign,274280.0,NaN,NaN,2101.0,0.470000,NaN,NaN,Detected,0.000878,NaN,Not evaluable,Not Detected,NaN,4.0,0.0,6/9/23,01/18/2023,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01902_0167_BH5VT7DSX7,/ghsfa/ivd/flowcentral/230629_A01902_0167_BH5VT7DSX7.8efb7722-560b-43a3-b3db-54cf45f51d00.20230711185414
1135,RMC-6236-001,02424040043,A0778829,B00426149,6236-001-088-0017,C1D1,1,SUCCESS,NaN,SNV,NaN,ULBP3,6,150390171.0,1,P11L,G>A,c.32C>T,NM_024518.3,53.54,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs377740812,NaN,NaN,NaN,NaN,1645.0,3.630000,NaN,NaN,Detected,0.001415,10.4,Low,Not Detected,NaN,3.5,0.0,6/9/23,10/05/2022,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01902_0167_BH5VT7DSX7,/ghsfa/ivd/flowcentral/230629_A01902_0167_BH5VT7DSX7.8efb7722-560b-43a3-b3db-54cf45f51d00.20230711185414
1143,RMC-6236-001,01731650681,A0778827,B00426151,6236-001-095-0070,C1D1,1,SUCCESS,NaN,SNV,NaN,RIF1,2,152276793.0,6,K198R,A>G,c.593A>G,NM_001177665.2,53.44,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2699.0,0.300000,NaN,NaN,Detected,0.002850,NaN,Not evaluable,Not Detected,NaN,3.5,0.0,6/9/23,04/20/2023,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01902_0167_BH5VT7DSX7,/ghsfa/ivd/flowcentral/230629_A01902_0167_BH5VT7DSX7.8efb7722-560b-43a3-b3db-54cf45f51d00.20230711185414
1142,RMC-6236-001,02431350022,A0778837,B00426152,6236-001-088-0067,C1D1,1,SUCCESS,NaN,SNV,NaN,FLT4,5,180030313.0,30,R1324P,C>G,c.3971G>C,NM_182925.4,52.37,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV56098908,rs307821,Likely_benign,251881.0,NaN,NaN,5051.0,27.922479,NaN,NaN,Detected,0.085692,29.3,High,Not Detected,NaN,4.0,0.0,6/9/23,03/30/2023,9/4/23,NaN,Not Applicable,Not Applicable,230629_A01907_0107_AH5VTLDSX7,/ghsfa/projects/omni_v2/flowcells/230629_A01907_0107_AH5VTLDSX7_PS3R388_AP3.Sirius-1.1.0-RLS_20230712-095904


In [ ]:
# Extract the "fc_dir" value
def extract_timestamp(folder):
    """
    Try to extract a timestamp from the folder name.
    Expected format: <runid>.<hash>.<timestamp>
    Timestamp format: YYYYMMDDHHMMSS.
    Returns a datetime object if successful, otherwise None.
    """
    # Match the pattern with a trailing dot and a 14-digit timestamp
    m = re.search(r'\.(\d{14})$', folder)
    if m:
        try:
            return datetime.strptime(m.group(1), '%Y%m%d%H%M%S')
        except ValueError:
            return None
    return None

def get_latest_candidate(folders, add_sirius=False):
    """
    Given a list of folder paths, resolve symlinks and check validity.
    Optionally check for a 'sirius' subfolder.
    Returns the folder path corresponding to the latest timestamp.
    """
    candidates = []
    for folder in folders:
        # Resolve symlink if necessary
        if os.path.islink(folder):
            resolved_folder = os.path.realpath(folder)
            if os.path.exists(resolved_folder):
                folder = resolved_folder
            else:
                continue  # Skip broken symlink

        if os.path.isdir(folder):
            candidate = folder
            if add_sirius:
                sirius_path = os.path.join(candidate, "sirius")
                if os.path.exists(sirius_path):
                    candidate = sirius_path
            # Try to extract the timestamp from the folder name
            ts = extract_timestamp(candidate)
            if ts is None:
                # Fall back: use modification time
                try:
                    mod_time = os.path.getmtime(candidate)
                    ts = datetime.fromtimestamp(mod_time)
                except Exception:
                    continue
            candidates.append((candidate, ts))
    
    if candidates:
        # Pick candidate with the maximum timestamp
        candidates.sort(key=lambda x: x[1], reverse=True)
        return candidates[0][0]
    return None

def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.
    It ensures only real, accessible directories are assigned and picks the latest candidate when multiple are found.
    
    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column.
    
    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column.
        pd.DataFrame: A DataFrame containing the rows for which no valid directory path was found.
    """
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"

    rows_no_found = []  # Stores rows with no valid directory found
    df['fc_dir'] = ''   # Initialize the column

    for index, row in df.iterrows():
        runid = row["runid"]
        valid_folder = None

        # Search in the first directory
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        valid_folder = get_latest_candidate(folders, add_sirius=True)

        # If not found in fc_dir_all, search in fc_dir_omni
        if not valid_folder:
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            valid_folder = get_latest_candidate(folders_omni, add_sirius=False)

        if valid_folder:
            df.loc[index, 'fc_dir'] = valid_folder
        else:
            rows_no_found.append(row)

    df_no_fc_dir = pd.DataFrame(rows_no_found)
    return df, df_no_fc_dir

# Generate the new dataframe 
df_2, df_no_found = add_fc_dir_column(df_1)

print("Total sample size of the Infinity batch with fc_dir is " + str(df_2.shape[0]) + "\n") 
print("Total sample size of the Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_2.head()


## Generate S3 MB caller outputs using Sirius 1.6.x yaml
###### Git clone "Sirius-1.6.0_Release_Branch" and "devel/mcai/update_crc_model_file" (Sirius-1.6.1) of bip-products for corresponding CRC caller cutoffs and correct CRC region_file for MB CRC caller
###### bip-epigenomics module instead of epi-caller was used for CRC result comparison

In [33]:
# Instead of using epicaller.methylation.methyl_caller funciton, bip-epigenomics module (branch 2.5) is prefered.

def run_bip_epigenomics_module_wrapper_slurm(sample_meta_df, workdir):
    module_dir = '/home/hazhang/repos/bip-epigenomics/'  # branch 2.5
    parameter_set_yaml = "/ghsfa/projects/pharma/customers/GuardantHealthIST/Infinity/MDC04_IST01_Infinity/MDC04_IST01_Infinity_01/data/rerun_crc_caller/bip-products/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # This is for Sirius 1.6.1
    # parameter_set_yaml = "/ghdevhome/home/hazhang/projects/Repo/bip-products_versions/bip-products-Sirius160/bip-products/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # This is for Sirius 1.6.0

    # Directories for jobs and output 
    jobs_dir = os.path.join(workdir, "MB_module_CRC_jobs")
    output_base_dir = os.path.join(workdir, "MB_module_CRC_output")
    
    # Create directories if they don't exist
    os.makedirs(jobs_dir, exist_ok=True)
    os.makedirs(output_base_dir, exist_ok=True)
    
    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]
        sample_dir = f"{fc_dir}/{run_sample_id}/"
        
        # Define input files from sample_dir
        mbd_hyper_molecules = os.path.join(sample_dir, f"{run_sample_id}.mbd_hyper_molecules.tsv")
        methyl_qc_file = os.path.join(sample_dir, f"{run_sample_id}.methyl_qc.tsv")
        subpanel_qc_file = os.path.join(sample_dir, f"{run_sample_id}.subpanel_qc_stats.hdr.tsv")
        
        # Skip sample if the main input file is missing
        if not os.path.exists(mbd_hyper_molecules):
            continue
        
        # Create output directory for the sample (mimics out_path/sampleID)
        out_dir = os.path.join(output_base_dir, run_sample_id)
        os.makedirs(out_dir, exist_ok=True)
        
        # Copy bip_config.json from fc_dir to the sample's output directory
        bip_config_json_file = os.path.join(fc_dir, "bip_config.json")
        os.system(f"cp {bip_config_json_file} {out_dir}")
        
        # Create a job file that will run all three steps sequentially
        job_file = os.path.join(jobs_dir, f"MB_call_{run_sample_id}.job")
        
        # Refined SLURM header
        shebang_content = (
            "#!/bin/bash\n"
            f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
            "#SBATCH --nodes=1\n"
            "#SBATCH --ntasks=1\n"
            "#SBATCH --cpus-per-task=16\n"
            "#SBATCH --mem=150g\n"
            "#SBATCH --partition=dev\n"
            f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n"
            f"#SBATCH --error={jobs_dir}/MB_call_{run_sample_id}.err\n\n"
        )
        
        # 1. run msre_caller
        cmd_msre_caller = (
            f"{module_dir}/module_entry msre_caller "
            f"--in-hyper-molecules {mbd_hyper_molecules} "
            f"--sample-id {run_sample_id} "
            f"--output-dir {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 16 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_msre_caller.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_msre_caller.log\n"
        )
        
        # 2. run epigenomics_tumor_fraction
        cmd_tumor_fraction = (
            f"{module_dir}/module_entry epigenomics_tumor_fraction "
            f"--in-hyper-molecules {mbd_hyper_molecules} "
            f"--in-methyl-qc-file {methyl_qc_file} "
            f"--in-subpanel-qc-file {subpanel_qc_file} "
            f"--sample-id {run_sample_id} "
            f"--output-dir {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 2 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_tumor_fraction.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_tumor_fraction.log\n"
        )
        
        # 3. run collect_methyl_callers
        cmd_collect = (
            f"{module_dir}/module_entry collect_methyl_callers "
            f"--in-msre-crc-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_crc_call.hdr.tsv "
            f"--in-msre-lung-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_lung_call.hdr.tsv "
            f"--in-msre-breast-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_breast_call.hdr.tsv "
            f"--in-msre-pancancer-call {out_dir}/{run_sample_id}/{run_sample_id}.msre_pancancer_call.hdr.tsv "
            f"--in-tf-estimate {out_dir}/{run_sample_id}/{run_sample_id}.epi_tf_model_predictions.tsv "
            f"--sample-id {run_sample_id} "
            f"--output {out_dir} "
            f"--parameter-set {parameter_set_yaml} "
            f"--threads 2 "
            f"--memory-log {out_dir}/{run_sample_id}-epigenomics_msre_summary.memory.log "
            f"--log-file {out_dir}/{run_sample_id}-epigenomics_msre_summary.log\n"
        )
        
        # Write all commands into the job file
        with open(job_file, "w") as jf:
            jf.write(shebang_content)
            jf.write(cmd_msre_caller)
            jf.write(cmd_tumor_fraction)
            jf.write(cmd_collect)
        
        # Submit the job file via sbatch
        os.system(f"sbatch {job_file}")


In [34]:
#run the function to generate all the MB S3 caller outputs
run_bip_epigenomics_module_wrapper_slurm(df_2, f"{work_dir}")

Submitted batch job 101339230
Submitted batch job 101339231
Submitted batch job 101339232
Submitted batch job 101339233
Submitted batch job 101339234
Submitted batch job 101339235
Submitted batch job 101339236
Submitted batch job 101339237


In [59]:
# Retrieve MB S3 CRC caller score and MB S3 Lung caller score based on the cancer type and stage
df_3 = df_2.copy()

df_3["MB_CRC_call_Sirius150"] = np.nan
df_3["MB_CRC_score_Sirius150"] = np.nan
df_3["pred_frac"] = np.nan

mb_caller_result_folder = f"{work_dir}/MB_module_CRC_output"
problematic_rows = []  # List to store indices of problematic rows

for index, crc_row in df_3.iterrows():
    run_sample_id = crc_row["GHSampleID"]
    msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}/{run_sample_id}/{run_sample_id}.msre_all_call.hdr.tsv"   
    try:
        msre_df = pd.read_table(msre_mr_file)
        msre_df_crc = msre_df[msre_df['model_name'] == 'CRC_LateStage']
        # print(msre_df_crc)

        if not msre_df_crc.empty:
            df_3.at[index, "pred_frac"] = msre_df_crc.iloc[0]["pred_frac"]
            df_3.at[index, "MB_CRC_score_Sirius150"] = msre_df_crc.iloc[0]["methyl_score"]
            df_3.at[index, "MB_CRC_call_Sirius150"] = "Detected" if msre_df_crc.iloc[0]["methyl_call"] == 1 else "Not Detected"
        else:
            # Add the index to the list if msre_df is empty
            problematic_rows.append(index) 
            
    except Exception as e:
        # Add the index to the list if there is any other exception
        problematic_rows.append(index)  
        print(f"Error processing file {msre_mr_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_3.loc[problematic_rows]

print("Total sample size of samples with NO MB output is " + str(df_problematic.shape[0]) + "\n") 

# Returning the DataFrame with problematic rows
df_3.head()


Total sample size of samples with NO MB output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_Sirius150,MB_CRC_score_Sirius150,pred_frac
2,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25,241221_A01020_0945_AHWTCMDSXC,/ghsfa/ivd/flowcentral/241221_A01020_0945_AHWTCMDSXC/sirius,Not Detected,-21.906225,0.000000
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC,/ghsfa/ivd/flowcentral/241227_A02019_0258_BHWTKCDSXC/sirius,Detected,-20.906050,0.000293
5,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF,/ghsfa/ivd/flowcentral/250114_A01021_0837_AH27KNDSXF/sirius,Detected,-20.515209,0.000101
4,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25,250125_A01744_0377_AH27JKDSXF,/ghsfa/ivd/flowcentral/250125_A01744_0377_AH27JKDSXF/sirius,Not Detected,-21.774074,0.000000
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF,/ghsfa/ivd/flowcentral/250126_A01021_0847_BH272FDSXF.0e716317-4529-4481-89c6-10be60fc1006.20250128124242/sirius,Detected,-20.986001,0.000250


## Check the result

In [8]:
df_flipped_call = df_3[df_3['MB_CRC_call_Sirius150'] != df_3['Tumor_methylation_status']]
print("Total CRC samples with flipped call is " + str(df_flipped_call.shape[0]) + "\n")

# Save the data
# df_flipped_call.to_csv(f"{work_dir}/20250121_Pfizer_SOW51core_Batch3_18Samples_InfinityReport_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20241220_GuardantHealthIST_IST01_Batch1_405Samples_InfinityReport_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20250120_Pfizer_SOW46Amendment1TP3_Batch9_20Samples_G360Report_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20250120_Pfizer_SOW46Amendment1TP5_Batch7_33Samples_InfinityReport_flipped_calls.csv", index = False)
df_flipped_call.to_csv(f"{work_dir}/XTX_monitoring_Sirius161_flipped_calls_by_cancer_stage_cutoff_03072025.csv", index = False)

df_flipped_call



Total CRC samples with flipped call is 5



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_Sirius150,MB_CRC_score_Sirius150,pred_frac
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC,/ghsfa/ivd/flowcentral/241227_A02019_0258_BHWTKCDSXC/sirius,Detected,-20.906050,0.000293
5,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF,/ghsfa/ivd/flowcentral/250114_A01021_0837_AH27KNDSXF/sirius,Detected,-20.515209,0.000101
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF,/ghsfa/ivd/flowcentral/250126_A01021_0847_BH272FDSXF.0e716317-4529-4481-89c6-10be60fc1006.20250128124242/sirius,Detected,-20.986001,0.000250
1,XTX101-0102-001,NaN,A1285830,A128583001,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,27.23,NaN,NaN,1/31/25,1/30/25,2/19/25,250202_A00569_0940_AH3CM5DSXF,/ghsfa/ivd/flowcentral/250202_A00569_0940_AH3CM5DSXF/sirius,Detected,-21.213370,0.000048
6,XTX101-0102-001,NaN,A1218823,B00848429,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,13.21,3.5,NaN,11/22/24,11/21/24,12/17/24,241204_A01112_0657_BHTFJKDSXC,/ghsfa/ivd/flowcentral/241204_A01112_0657_BHTFJKDSXC/sirius,Detected,-21.248167,0.000100


## Generate the epiTFv2 gamma score results

In [60]:
# Temporary solution for TFv2gamma score generation
df_sow202_3 = df_3[df_3['Sample_status'] != "FAIL"]
df_sow202_3.shape

# Save the data
df_sow202_3.to_csv(f"{work_dir}/df_sow202_3.csv", index = False)

In [50]:
# Step 1: save the input csv file
df_sow202_3 = pd.read_csv(f"{work_dir}/df_sow202_3.csv", header = 0)
df_sow202_3.head()

# Step 2 generate the slurm script
TFv2gamma_output_dir = f"{work_dir}/TFv2gamma_SOW202_output"

# Check and create directories if not exists
if not os.path.exists(TFv2gamma_output_dir):
    os.makedirs(TFv2gamma_output_dir)

In [38]:
# Step 2 generate the slurm script

# Define the contents of the slurm script
# Parts to updates
    # sample_list; out_path; slurm_script_path

slurm_script_content = f"""#! /bin/sh
#SBATCH --get-user-env
#SBATCH --job-name TFv2gamma_pancanTF
#SBATCH --partition dev
#SBATCH --mem-per-cpu 40G
#SBATCH --cpus-per-task 1
#SBATCH --ntasks-per-node 1
#SBATCH -o {TFv2gamma_output_dir}/epiTFv2gamma_%A.log
#SBATCH --array 1-1000%300

module load python/3.9.6;
export INSTALL_DIR=/home/mcai/workspace/tfv2_gamma/bip-pycommon;
source $INSTALL_DIR/.py3_venv/bin/activate;
export PYTHONPATH=$INSTALL_DIR:$PYTHONPATH;

sample_list={work_dir}/df_sow202_3.csv
out_path="{TFv2gamma_output_dir}"
mkdir -p $out_path

# Use Python to extract the necessary fields robustly
# Note that the parsing method is changed due to weired parsing issue of "fc_dir" of this sow
read sample_id fc_dir <<< $(python -c "import pandas as pd; import os
df = pd.read_csv('$sample_list')
# Adjusting SLURM_ARRAY_TASK_ID from 1-based to 0-based index
index = int(os.getenv('SLURM_ARRAY_TASK_ID', '1')) - 1
sample_id = df.iloc[index]['GHSampleID']
fc_dir = df.iloc[index]['fc_dir']
print(sample_id, fc_dir)")

# Prepend base path if fc_dir is relative
base_path="/ghsfa/ivd/flowcentral"
if [[ "$fc_dir" != /* ]]; then
    fc_dir="${{base_path}}/${{fc_dir}}"
fi

# Construct paths
sample_path="${{fc_dir}}/${{sample_id}}"
mbd_path="${{sample_path}}/${{sample_id}}.mbd_hyper_molecules.tsv"
methyl_qc_file="${{sample_path}}/${{sample_id}}.methyl_qc.tsv"
subpanel_qc_path="${{mbd_path/mbd_hyper_molecules/subpanel_qc_stats.hdr}}"

# Run the code
python -m epi_tumor_fraction.run_epi_tf $mbd_path $methyl_qc_file $out_path $sample_id /ghdevhome/home/mcai/workspace/tfv2_gamma/bip-products/models/Sirius/epi_tf_models/params.sirius.yaml --subpanel_qc_file $subpanel_qc_path
"""

# Save the slurm script to file
slurm_script_path = f"{work_dir}/run_TFv2gamma_sow202.sh"
with open(slurm_script_path, "w") as file:
    file.write(slurm_script_content)

In [39]:
# Step 3 run the script
os.system(f"sbatch {work_dir}/run_TFv2gamma_sow202.sh")

Submitted batch job 101339245


0

In [61]:
# Retrive the TFv2gamma value

# Remove the "FAIL" samples
df_sow202_4 = df_sow202_3[df_sow202_3["Sample_status"] != "FAIL"]

# Initialize a list to store rows where no TFv2gamma output was found
rows_no_tfv2 = []

df_sow202_4['pred_frac_force'] = None 

TFv2gamma_output_dir = f"{work_dir}/TFv2gamma_SOW202_output"

for index, row in df_sow202_4.iterrows():
    sample_id = row["GHSampleID"]
    file_path = f"{TFv2gamma_output_dir}/{sample_id}.epi_tf_model_predictions.tsv"

    file_path_tfv2 = glob.glob(file_path)
    if not file_path_tfv2:
        rows_no_tfv2.append(row)
    
    else:
        try:
            df_temp = pd.read_table(file_path)
            value_gamma = df_temp[df_temp['metric'] == 'epi_tf']['value'].values[0]
            df_sow202_4.at[index, 'pred_frac_force'] = value_gamma
            
        except Exception as e:
            print(f"An error occurred with {sample_id}: {str(e)}")

# Create a DataFrame with rows where folders were not found in any sources
df_no_tfv2 = pd.DataFrame(rows_no_tfv2)
print("Total sample size of SOW202 Infinity cumulative batches with NO TFv2gamma output is " + str(df_no_tfv2.shape[0]) + "\n") 


df_sow202_5 = df_sow202_4.copy()
df_sow202_5.head()

Total sample size of SOW202 Infinity cumulative batches with NO TFv2gamma output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_Sirius150,MB_CRC_score_Sirius150,pred_frac,pred_frac_force
2,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25,241221_A01020_0945_AHWTCMDSXC,/ghsfa/ivd/flowcentral/241221_A01020_0945_AHWTCMDSXC/sirius,Not Detected,-21.906225,0.000000,0.000221
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC,/ghsfa/ivd/flowcentral/241227_A02019_0258_BHWTKCDSXC/sirius,Detected,-20.906050,0.000293,0.000402
5,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF,/ghsfa/ivd/flowcentral/250114_A01021_0837_AH27KNDSXF/sirius,Detected,-20.515209,0.000101,0.000158
4,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25,250125_A01744_0377_AH27JKDSXF,/ghsfa/ivd/flowcentral/250125_A01744_0377_AH27JKDSXF/sirius,Not Detected,-21.774074,0.000000,0.000106
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF,/ghsfa/ivd/flowcentral/250126_A01021_0847_BH272FDSXF.0e716317-4529-4481-89c6-10be60fc1006.20250128124242/sirius,Detected,-20.986001,0.000250,0.00045


In [ ]:
df_sow202_5

In [62]:
df_sow202_5['diff'] = df_sow202_5['pred_frac_force'] - df_sow202_5['pred_frac'] 
df_sow202_5

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_Sirius150,MB_CRC_score_Sirius150,pred_frac,pred_frac_force,diff
2,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25,241221_A01020_0945_AHWTCMDSXC,/ghsfa/ivd/flowcentral/241221_A01020_0945_AHWTCMDSXC/sirius,Not Detected,-21.906225,0.000000,0.000221,0.000221
3,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC,/ghsfa/ivd/flowcentral/241227_A02019_0258_BHWTKCDSXC/sirius,Detected,-20.906050,0.000293,0.000402,0.000109
5,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF,/ghsfa/ivd/flowcentral/250114_A01021_0837_AH27KNDSXF/sirius,Detected,-20.515209,0.000101,0.000158,0.000057
4,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25,250125_A01744_0377_AH27JKDSXF,/ghsfa/ivd/flowcentral/250125_A01744_0377_AH27JKDSXF/sirius,Not Detected,-21.774074,0.000000,0.000106,0.000106
0,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF,/ghsfa/ivd/flowcentral/250126_A01021_0847_BH272FDSXF.0e716317-4529-4481-89c6-10be60fc1006.20250128124242/sirius,Detected,-20.986001,0.000250,0.00045,0.0002
1,XTX101-0102-001,NaN,A1285830,A128583001,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,27.23,NaN,NaN,1/31/25,1/30/25,2/19/25,250202_A00569_0940_AH3CM5DSXF,/ghsfa/ivd/flowcentral/250202_A00569_0940_AH3CM5DSXF/sirius,Detected,-21.213370,0.000048,0.000109,0.000062
6,XTX101-0102-001,NaN,A1218823,B00848429,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,13.21,3.5,NaN,11/22/24,11/21/24,12/17/24,241204_A01112_0657_BHTFJKDSXC,/ghsfa/ivd/flowcentral/241204_A01112_0657_BHTFJKDSXC/sirius,Detected,-21.248167,0.000100,0.000167,0.000067


In [18]:
df_sow202_5

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_Sirius150,MB_CRC_score_Sirius150,pred_frac,pred_frac_force
0,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25,241221_A01020_0945_AHWTCMDSXC,/ghsfa/ivd/flowcentral/241221_A01020_0945_AHWTCMDSXC/sirius,Not Detected,-21.906225,0.000000,0.000221
1,XTX101-0102-001,NaN,A1247604,A124760401,146-005,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,26.64,NaN,NaN,12/21/24,12/19/24,1/2/25,241227_A02019_0258_BHWTKCDSXC,/ghsfa/ivd/flowcentral/241227_A02019_0258_BHWTKCDSXC/sirius,Detected,-20.906050,0.000293,0.000402
2,XTX101-0102-001,NaN,A1263902,A126390201,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,11.10,NaN,NaN,1/11/25,1/9/25,2/4/25,250114_A01021_0837_AH27KNDSXF,/ghsfa/ivd/flowcentral/250114_A01021_0837_AH27KNDSXF/sirius,Detected,-20.515209,0.000101,0.000158
3,XTX101-0102-001,NaN,A1276510,A127651001,140-008,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Not Detected,Not Detected,0,33.94,NaN,NaN,1/23/25,1/22/25,2/11/25,250125_A01744_0377_AH27JKDSXF,/ghsfa/ivd/flowcentral/250125_A01744_0377_AH27JKDSXF/sirius,Not Detected,-21.774074,0.000000,0.000106
4,XTX101-0102-001,NaN,A1277545,A127754501,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,19.86,NaN,NaN,1/24/25,1/23/25,2/11/25,250126_A01021_0847_BH272FDSXF,/ghsfa/ivd/flowcentral/250126_A01021_0847_BH272FDSXF.0e716317-4529-4481-89c6-10be60fc1006.20250128124242/sirius,Detected,-20.986001,0.000250,0.00045
5,XTX101-0102-001,NaN,A1285830,A128583001,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,27.23,NaN,NaN,1/31/25,1/30/25,2/19/25,250202_A00569_0940_AH3CM5DSXF,/ghsfa/ivd/flowcentral/250202_A00569_0940_AH3CM5DSXF/sirius,Detected,-21.213370,0.000048,0.000109
6,XTX101-0102-001,NaN,A1218823,B00848429,141-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Not Detected,Not Detected,0,13.21,3.5,NaN,11/22/24,11/21/24,12/17/24,241204_A01112_0657_BHTFJKDSXC,/ghsfa/ivd/flowcentral/241204_A01112_0657_BHTFJKDSXC/sirius,Detected,-21.248167,0.000100,0.000167
